# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [xju2.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2009-10-01	Paper Title Number 1	Journal 1	This paper is about the number 1. The number 2 is left for future work.	Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).	paper-title-number-1	http://academicpages.github.io/files/paper1.pdf
2010-10-01	Paper Title Number 2	Journal 1	This paper is about the number 2. The number 3 is left for future work.	Your Name, You. (2010). "Paper Title Number 2." <i>Journal 1</i>. 1(2).	paper-title-number-2	http://academicpages.github.io/files/paper2.pdf
2015-10-01	Paper Title Number 3	Journal 1	This paper is about the number 3. The number 4 is left for future work.	Your Name, You. (2015). "Paper Title Number 3." <i>Journal 1</i>. 1(3).	paper-title-number-3	http://academicpages.github.io/files/paper3.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [11]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications

,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2010-12-14,Measurement of the W &rarr; lv and Z &rarr; ll...,JHEP,NaN,"ATLAS Collaboration, JHEP 1012 (2010) 060",p,https://link.springer.com/article/10.1007/JHEP...
1,2011-11-24,Search for the Standard Model Higgs boson in t...,Phys.Lett.B,NaN,"ATLAS Collaboration, Phys.Lett.B 705 (2011) 43...",p,http://www.sciencedirect.com/science/article/p...
2,2012-04-12,Search for the Standard Model Higgs boson in t...,Phys.Lett.B,NaN,Phys.Lett.B 710 (2012) 383-402,p,http://www.sciencedirect.com/science/article/p...
3,2012-03-29,Combined search for the Standard Model Higgs b...,Phys.Lett.B,NaN,"ATLAS Collaboration, Phys.Lett.B 710 (2012) 49-66",p,http://www.sciencedirect.com/science/article/p...
4,2012-12-21,A Particle Consistent with the Higgs Boson Obs...,Science,NaN,"ATLAS Collaboration, Science 338 1576-1582",p,http://science.sciencemag.org/content/338/6114...
5,2012-09-17,Observation of a new particle in the search fo...,Phys.Lett.B,NaN,"ATLAS Collaboration, Phys.Lett.B 716 (2012) 1-29",p,http://www.sciencedirect.com/science/article/p...
6,2014-09-09,Measurement of the Higgs boson mass from the H...,Phys.Rev.D,NaN,"ATLAS Collaboration, Phys.Rev.D 90 (2014) 052004",p,https://journals.aps.org/prd/abstract/10.1103/...
7,2015-01-16,Measurement of Higgs boson production and coup...,Phys.Rev.D,NaN,"Phys.rev.D 91 (2015) no.1, 012006",p,https://journals.aps.org/prd/abstract/10.1103/...
8,2016-01-05,Measurements of the Higgs boson production and...,EPJC,NaN,Eur.Phys.J. C 76 (2016) no.1,p,https://link.springer.com/article/10.1140%2Fep...
9,2016-08-22,Search for new phenomena in final states with ...,Phys.Rev.D,NaN,"Phys.Rev.D 94 (2016) no.3, 032005",p,https://journals.aps.org/prd/abstract/10.1103/...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [6]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    if not pd.isnull(text):
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return " "

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [7]:
import os
idx_row = 0
for row, item in publications.iterrows():
    idx_row += 1
    md_filename = str(item.pub_date) + "-" + item.url_slug + str(idx_row)+ ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug + str(idx_row)
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + str(item.citation)
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

TypeError: Can't convert 'float' object to str implicitly

These files are in the publications directory, one directory below where we're working from.

In [26]:
!ls ../_publications/

2010-12-14-p1.md  2012-09-17-p6.md  2016-01-05-p9.md  2017-07-01-p12.md
2011-11-24-p2.md  2012-12-21-p5.md  2016-08-22-p10.md 2017-10-01-p13.md
2012-03-29-p4.md  2014-09-09-p7.md  2016-09-01-p11.md
2012-04-12-p3.md  2015-01-16-p8.md  2017-07-01-j6.md


In [15]:
!cat ../_publications/2011-09-p2.md

---
title: "Search for the Standard Model Higgs boson in the decay channel H->ZZ->4l with the ATLAS detector"
collection: publications
permalink: /publication/2011-09-p2
date: 2011-09
venue: 'Phys.Lett.B'
paperurl: 'http://inspirehep.net/record/929699'
citation: 'Phys.Lett.B 705 (2011) 435-451'
---
[Download paper here](http://inspirehep.net/record/929699)

Recommended citation: Phys.Lett.B 705 (2011) 435-451